In [12]:
!pip install SpeechRecognition

In [13]:
pip install deepface


In [14]:
pip install moviepy SpeechRecognition pydub

In [15]:
import cv2  # OpenCV
import numpy as np
from keras.models import load_model  # For CNN model
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pandas as pd
from deepface import DeepFace
import moviepy.editor as mp
import speech_recognition as sr

In [16]:

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Input video
video_path = '/content/3.mp4' # <-- Replace with your video file
cap = cv2.VideoCapture(video_path)

# Create folder to save face images
output_folder = 'detected_faces'
os.makedirs(output_folder, exist_ok=True)

frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process every 10th frame (to reduce load)
    if frame_count % 10 == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for i, (x, y, w, h) in enumerate(faces):
            face_img = frame[y:y+h, x:x+w]
            face_filename = os.path.join(output_folder, f'face_{frame_count}_{i}.jpg')
            cv2.imwrite(face_filename, face_img)
            saved_count += 1

    frame_count += 1

cap.release()
print(f"Done! Total faces saved: {saved_count}")


Done! Total faces saved: 39


In [17]:
# import os
# import cv2
# import pandas as pd
# from deepface import DeepFace

# Folder with face images
face_folder = 'detected_faces'

# Loop through images
for img_file in os.listdir(face_folder):
    if img_file.endswith('.jpg'):
        img_path = os.path.join(face_folder, img_file)
        try:
            # Read image
            img = cv2.imread(img_path)

            # Analyze emotion
            result = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)

            # Get predicted emotion
            emotion = result[0]['dominant_emotion']
            print(f"{img_file}: {emotion}")

        except Exception as e:
            print(f"Error analyzing {img_file}: {e}")

        results = []

        # Inside the loop:
        results.append({'Image': img_file, 'Emotion': emotion})

# After the loop:
df = pd.DataFrame(results)
df.to_csv('emotion_results.csv', index=False)



face_310_1.jpg: neutral
face_820_0.jpg: fear
face_630_0.jpg: angry
face_840_0.jpg: angry
face_640_0.jpg: angry
face_600_0.jpg: angry
face_50_0.jpg: neutral
face_260_0.jpg: angry
face_580_0.jpg: angry
face_0_0.jpg: angry
face_20_0.jpg: neutral
face_650_0.jpg: happy
face_60_0.jpg: neutral
face_420_0.jpg: sad
face_430_0.jpg: neutral
face_60_1.jpg: neutral
face_360_0.jpg: happy
face_850_0.jpg: angry
face_40_0.jpg: neutral
face_790_0.jpg: fear
face_30_0.jpg: neutral
face_440_0.jpg: angry
face_800_0.jpg: fear
face_530_0.jpg: angry
face_930_0.jpg: fear
face_250_0.jpg: angry
face_930_1.jpg: neutral
face_780_0.jpg: surprise
face_590_0.jpg: surprise
face_450_0.jpg: angry
face_10_0.jpg: neutral
face_680_0.jpg: angry
face_560_0.jpg: sad
face_370_0.jpg: neutral
face_810_0.jpg: sad
face_340_0.jpg: angry
face_350_0.jpg: fear
face_660_0.jpg: angry
face_920_0.jpg: fear
face_710_0.jpg: angry
face_570_0.jpg: angry
face_310_0.jpg: neutral
face_550_0.jpg: angry


In [20]:
# import moviepy.editor as mp
# import speech_recognition as sr

# Extract audio from video
video = mp.VideoFileClip("/content/3.mp4")
video.audio.write_audiofile("audio.wav")

# Convert audio to text
recognizer = sr.Recognizer()
with sr.AudioFile("audio.wav") as source:
    audio = recognizer.record(source)

try:
    text = recognizer.recognize_google(audio)
    print("Transcribed text:\n", text)
except sr.UnknownValueError:
    print("Could not understand the audio.")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

train_texts = [
    "I am feeling very happy today",
    "This is so sad and painful",
    "I am extremely angry right now",
    "What a wonderful day",
    "I'm so scared of what might happen",
    "Everything is boring and dull",
    "I feel great and joyful",
    "I can't stop crying",
    "Why are you yelling at me!",
    "I'm trembling with fear",
    "Nothing excites me anymore"
]
train_labels = ['happy', 'sad', 'angry', 'happy', 'fear', 'neutral', 'happy', 'sad', 'angry', 'fear', 'neutral']


# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)

# Train SVM
svm = SVC(kernel='linear')
svm.fit(X_train, train_labels)

# Use transcribed video text for prediction
video_text = text  # From the earlier step
X_test = vectorizer.transform([video_text])
predicted_emotion = svm.predict(X_test)

print("Predicted emotion from audio/text:", predicted_emotion[0])


MoviePy - Writing audio in audio.wav


MoviePy - Done.
Transcribed text:
 how to System of a Down in 30 seconds drop C or C sharp tuning half steps thrush it up Groove hi grocery list lyrics death metal acoustic random go tree go
Predicted emotion from audio/text: fear


In [25]:
import cv2
from deepface import DeepFace
from collections import Counter
import os

# Create folder to store frames
os.makedirs("frames", exist_ok=True)

# Load video and extract frames
# Changed video path to the correct one, assuming it's the same as before
cap = cv2.VideoCapture("/content/3.mp4")
frame_rate = 30  # analyze one frame every 30 frames (~1 second if 30fps)
frame_id = 0
saved_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_id % frame_rate == 0:
        frame_path = f"frames/frame_{frame_id}.jpg"
        cv2.imwrite(frame_path, frame)
        saved_frames.append(frame_path)
    frame_id += 1

cap.release()

# Analyze each saved frame using DeepFace for emotion
face_emotions = {}
for path in saved_frames:
    try:
        result = DeepFace.analyze(img_path=path, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
        face_emotions[os.path.basename(path)] = emotion
        print(f"{path} → {emotion}")
    except Exception as e:
        print(f"Skipping {path}: {e}")

# Count most common face emotion
face_counts = Counter(face_emotions.values())

# Check if face_counts is empty before accessing most_common
if face_counts:
    most_common_face_emotion, face_votes = face_counts.most_common(1)[0]
    print(f"Most common facial emotion: {most_common_face_emotion} ({face_votes} votes)")
else:
    print("No faces detected or analyzed for emotions in the video.")

frames/frame_0.jpg → angry
frames/frame_30.jpg → neutral
frames/frame_60.jpg → surprise
frames/frame_90.jpg → fear
frames/frame_120.jpg → angry
frames/frame_150.jpg → angry
frames/frame_180.jpg → sad
frames/frame_210.jpg → sad
frames/frame_240.jpg → angry
frames/frame_270.jpg → sad
frames/frame_300.jpg → happy
frames/frame_330.jpg → neutral
frames/frame_360.jpg → happy
frames/frame_390.jpg → neutral
frames/frame_420.jpg → sad
frames/frame_450.jpg → angry
frames/frame_480.jpg → angry
frames/frame_510.jpg → surprise
frames/frame_540.jpg → angry
frames/frame_570.jpg → angry
frames/frame_600.jpg → angry
frames/frame_630.jpg → sad
frames/frame_660.jpg → angry
frames/frame_690.jpg → sad
frames/frame_720.jpg → sad
frames/frame_750.jpg → neutral
frames/frame_780.jpg → surprise
frames/frame_810.jpg → sad
frames/frame_840.jpg → angry
frames/frame_870.jpg → sad
frames/frame_900.jpg → sad
frames/frame_930.jpg → fear
frames/frame_960.jpg → happy
Most common facial emotion: angry (11 votes)
frames/f